<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="./images/btp-banner.gif" alt="BTP A&C">
</div>

## Retrieval-Augmented Generation with SAP HANA Cloud Vector Engine

In this demo, we will explore how to build **Knowledge Graphs (KGs)** from **tabular data**. Specifically, we will use tables **S013** and **LFA1** from the **SPURCHASE** schema to construct an **ontology** that captures the semantic relationships between these tables. The ontology will define key aspects such as table relationships, join conditions, and relevant columns. This semantic layer serves as the foundation for generating knowledge graphs, enabling more meaningful data integration and querying. Once the ontology is created, we will import it into **SAP HANA Cloud** to generate the Knowledge Graphs. The import can be done either via a local file or through supported cloud storage options. 

With the ontology and knowledge graph in place, we aim to **answer complex business or analytical questions** by leveraging the ontologies created and stored within the **SAP HANA Cloud database**. These ontologies provide a **semantic layer** that describes the structure, meaning, and relationships of the data in a machine-readable format, enabling more intelligent and context-aware data access. By using this semantic model, we can translate user queries or application logic into meaningful insights, even when the data resides in multiple interrelated tables. This approach enhances data **discovery, reasoning, and integration**, and supports more intuitive querying through graph-based models rather than complex joins or manual interpretation of raw table structures.

## 🎯Learning Objectives
By the end of this demo, you will be able to:
- Design and define ontologies using RDF/OWL standards to model business entities and their relationships.
- Construct and serialize RDF graphs using Python libraries (e.g., rdflib) and export them in Turtle (.ttl) format.
- Load and persist knowledge graphs into SAP HANA Cloud and validate their structure using SPARQL queries.
- Perform semantic retrieval by writing SPARQL queries to extract relevant metadata based on user intent.
- Map natural language questions to structured data by analyzing intent and aligning with metadata semantics.
- Generate SQL queries dynamically using insights from the knowledge graph to answer complex business questions.

## 🚨Requirements

Please review the following requirements before starting the demo: 
- Enable the additional feature **Triple Store** in your SAP HANA Cloud database 
- Deploy Large Language Models (LLMs): **anthropic--claude-3.5-sonnet** in SAP AI Launchpad

## 📝About the Data
This Dataset is a **simulated version of vendor data** from **SAP S/4HANA**, created for demonstration purposes. It includes **vendor evaluation records (S013)** and **supplier master data (LFA1)**, which are essential for assessing vendor performance, identifying potential supply chain risks, and supporting data-driven procurement decisions.

For the demo calculation of risk score on table S013, we used the following weighting key (sums to 1.00):
|Criterion (column)	|Description	|Weight |
|-------------------|---------------|-------|
|**PWMT1**	|Quantity-reliability points	|0.15|
|**PWTT1**	|On-time-delivery points	|0.15|
|**PWEV1**	|Compliance with shipping instructions	|0.10|
|**PWWE1**	|Service-quality points	|0.15|
|**PWFR1**	|Service timeliness	|0.10|
|**PWQA1**	|Quality-audit points	|0.10|
|**LAVI1**	|Shipping-notification performance	|0.15|
|**ALAV1**	|Variance from shipping notification	|0.10|
|**Total**	|—	|1.00|

- **VENDOR_SCORE** = Σ( score × weight ) / 1.00  
- **RISK_SCORE**   = 100 – VENDOR_SCORE

> **Important**: 
> - These weights are just an example. In a real SAP system they must mirror the weighting key defined in SPRO ▸ Materials Management ▸ Purchasing ▸ Vendor Evaluation ▸ Define Weighting Keys.
> - If your system uses the standard key 000 (equal importance) you would set each weight to 0.125 (12.5 %) or simply average the eight criteria.
> - Management dashboards often base the overall vendor rating on the “…2” values to avoid wild month-to-month swings.
> - If you load true historical S013 data (several periods) into HANA, switching the formula to the “…2” fields will give exactly the same logic SAP uses in the standard report.

### Step 1: Install Python Packages  
Run the following package installations. **pip** is the package installer for Python. You can use pip to install packages from the Python Package Index and other indexes. 

⚠️**Note:** Jupyter Notebook kernel restart required after package installation. 

In [ ]:
%pip install rdflib --break-system-packages
%pip install hdbcli --break-system-packages
%pip install generative-ai-hub-sdk[all] --break-system-packages
%pip install langchain-core --break-system-packages
%pip install pandas --break-system-packages
%pip install xml.etree --break-system-packages
%pip install python-dotenv --break-system-packages
# kernel restart required!!!

### Step 2: Initialize the RDF graph and add table metadata into the graph
This code snippet demonstrates how to initialize an RDF graph using a Python library such as RDFLib, and how to construct the graph by extracting metadata from database tables, such as table names, column names, data types, and relationships (e.g., foreign key constraints). This metadata is then translated into RDF triples, capturing the structure and semantics of the underlying tabular data. The resulting RDF graph serves as the foundation for building ontologies or knowledge graphs, enabling semantic reasoning and enhanced data integration.

In [ ]:
from rdflib import Graph, Literal, Namespace, RDF, RDFS, XSD

# Define custom namespaces for our RDF graph
ns = Namespace("http://supplychain_database.org/spurchase/")  # Namespace for supply chain data
db = Namespace("http://supplychain_database.org/database/")  # Namespace for database schema

# Create an empty RDF graph
g = Graph()

# Bind namespace prefixes for cleaner serialization
g.bind("spurchase", ns)  # Associates "spurchase" prefix with our namespace
g.bind("db", db)       # Associates "db" prefix with database namespace

# Define table resources
s013 = ns.S013      # Resource for vendor evaluation table
lfa1 = ns.LFA1  # Resource for vender metadata table

# Add metadata for S013 table
g.add((s013, RDF.type, db.Table))            # Set type as Table
g.add((s013, RDFS.label, Literal("Vendor Risk Evaluations")))  # Human-readable label
g.add((s013, db.tableName, Literal("S013"))) # Actual table name in database

# Add metadata for LFA1 table
g.add((lfa1, RDF.type, db.Table))            # Set type as Table
g.add((lfa1, RDFS.label, Literal("Vendor Details")))  # Human-readable label
g.add((lfa1, db.tableName, Literal("LFA1"))) # Actual table name in database

print("Your knowledge graph is initialized!✅")

### Step 3: Define columns and metadata for Table S013
The following code snippet constructs an RDF graph by extracting metadata from the database table S013, which is typically used for storing vendor evaluation data. The extracted metadata includes column names, column descriptions, applicable aggregation methods, and relationships to other tables, such as those defined by foreign key constraints. This metadata is then transformed into RDF triples, where each piece of information is represented as a semantic statement. By doing so, the graph captures not only the structure but also the contextual meaning of the data within S013, forming an essential component of a broader knowledge graph for enhanced data integration, reasoning, and analysis.

In [ ]:
# Define columns and metadata for S013 table
s013_columns = {
    # Client column metadata
    ns.MANDT: {
        "label": "Client",
        "isKey": True,  # Mark as primary key
        "description": "Client identifier"
    },
    # Vendor ID column metadata (foreign key)
    ns.LIFNR: {
        "label": "Vendor ID",
        "isKey": True,  # Part of composite key
        "foreignKey": ns.LIFNR,  # References LFA1.LIFNR
        "groupBy": True,  # Can be used for grouping
        "description": "Foreign key to LFA1.LIFNR"
    },
    # Material Number column metadata
    ns.MATNR: {
        "label": "Material Number",
        "isKey": True,  # Part of composite key
        "groupBy": True,  # Can be used for grouping
        "description": "Product identifier"
    },
    # Month column metadata
    ns.SPMON: {
        "label": "Month",
        "groupBy": True,  # Can be used for grouping
        "description": "Month to analyze (NVARCHAR format)"
    },
    # Purchasing Organization column metadata
    ns.EKORG: {
        "label": "Purchasing Organization",
        "aggregation": db.COUNT,  # Can be used with COUNT function
        "description": "Organization identifier"
    },
    # Quantity Reliability column metadata
    ns.PWMT1: {
        "label": "Quantity Reliability Score",
        "aggregation": db.AVG,  # Can be used with AVG function
        "description": "Points score (1-100) for quantity reliability"
    },
    # On-time Delivery column metadata
    ns.PWTT1: {
        "label": "On-time Delivery Score",
        "aggregation": db.AVG,  # Can be used with AVG function
        "description": "Points score (1-100) for on-time delivery performance"
    },
    # Shipping Compliance column metadata
    ns.PWEV1: {
        "label": "Shipping Compliance Score",
        "aggregation": db.AVG,  # Can be used with AVG function
        "description": "Points score (1-100) for compliance with shipping instructions",
    },
    # Service Quality column metadata
    ns.PWWE1: {
        "label": "Service Quality Score",
        "aggregation": db.AVG,  # Can be used with AVG function
        "description": "Points score (1-100) for service quality",
    },
    # Service Timeliness column metadata
    ns.PWFR1: {
        "label": "Service Timeliness Score",
        "aggregation": db.AVG,  # Can be used with AVG function
        "description": "Points score (1-100) for service timeliness",
    },
    # Quality Audit column metadata
    ns.PWQA1: {
        "label": "Quality Audit Score",
        "aggregation": db.AVG,  # Can be used with AVG function
        "description": "Points score (1-100) for quality audit",
    },
    # Shipping Notification column metadata
    ns.LAVI1: {
        "label": "Shipping Notification Score",
        "aggregation": db.AVG,  # Can be used with AVG function
        "description": "Points score (1-100) for shipping notification",
    },
    # Variance from Shipping Notification column metadata
    ns.ALAV1: {
        "label": "Variance from Shipping Notification Score",
        "aggregation": db.AVG,  # Can be used with AVG function
        "description": "Points score (1-100) for variance from shipping notification",
    },
    # Vendor Reliability column metadata
    ns.RELIA: {
        "label": "Vendor Reliability Score",
        "aggregation": db.AVG,  # Can be used with AVG function
        "description": "Vendor reliability score (1-100)",
    },
    # Vendor Risk column metadata
    ns.RISK1: {
        "label": "Vendor Risk Score",
        "aggregation": db.AVG,  # Can be used with AVG function
        "description": "Vendor risk score (1-100)",
    }
}

# Add all S013 columns to the graph
for col, meta in s013_columns.items():
    # Basic column metadata
    g.add((col, RDF.type, db.Column))  # Set type as Column
    g.add((col, RDFS.label, Literal(meta["label"])))  # Human-readable label
    g.add((col, db.columnName, Literal(col.split("/")[-1])))  # Extract column name from URI
    g.add((col, db.description, Literal(meta["description"])))  # Description

    # Conditional metadata additions
    if meta.get("isKey"):
        g.add((col, db.isPrimaryKey, Literal(True)))  # Mark as primary key
    if meta.get("groupBy"):
        g.add((col, db.groupBy, Literal(True)))  # Mark as groupable
    if meta.get("aggregation"):
        g.add((col, db.aggregationFunction, meta["aggregation"]))  # Add aggregation function
    # if meta.get("filter"):
    #    g.add((col, db.filterFunction, Literal(meta["filter"])))  # Add filter function
    if meta.get("foreignKey"):
        g.add((col, db.foreignKey, meta["foreignKey"]))  # Add foreign key reference

print("✅Table S013 has been added to your knowledge graph.")
print(f"👍There are {len(g)} triples in your knowledge graph.")

### Step 4: Define columns and metadata for Table LFA1
The following code snippet constructs an RDF graph by programmatically extracting metadata from the database table LFA1, which typically contains vendor master data. This metadata includes details such as column names, column descriptions, data types, potential aggregation rules.

In [ ]:
# Define columns and metadata for LFA1 table
lfa1_columns = {
    # Vendor ID column metadata (foreign key)
    ns.LIFNR: {
        "label": "Vendor ID",
        "isKey": True,  # Primary key
        "description": "Primary key for vendor"
    },
    # Vendor name column metadata
    ns.NAME1: {
        "label": "Vendor Name",
        "description": "Full name of vendor"
    },
    # Country Key column metadata
    ns.LAND1: {
        "label": "Country Key",
        "description": "Country code of vendor (ISO 3166-1 alpha-2)"
    },
    # City column metadata
    ns.ORT01: {
        "label": "City",
        "description": "City of vendor"
    },
    # Address column metadata
    ns.STRAS: {
        "label": "Vendor Address",
        "description": "House number and street name of vendor"
    }
}

# Add all LFA1 columns to the graph
for col, meta in lfa1_columns.items():
    # Basic column metadata
    g.add((col, RDF.type, db.Column))  # Set type as Column
    g.add((col, RDFS.label, Literal(meta["label"])))  # Human-readable label
    g.add((col, db.columnName, Literal(col.split("/")[-1])))  # Extract column name from URI
    g.add((col, db.description, Literal(meta["description"])))  # Description
    # Conditional metadata additions
    if meta.get("isKey"):
        g.add((col, db.isPrimaryKey, Literal(True)))  # Mark as primary key

print("✅Table LFA1 has been added to your knowledge graph.")
print(f"👍There are {len(g)} triples in your knowledge graph.")

### Step 5: Define relationships between tables
The following code snippet constructs an RDF graph by explicitly defining the semantic relationships between the two database tables, S013 and LFA1. It establishes how these tables are linked—typically through foreign keys or logical associations—and represents these connections using RDF triples. Each triple encodes a subject-predicate-object relationship, allowing the graph to capture the meaning and structure of the data. This forms the basis for integrating relational data into a knowledge graph, enabling advanced querying, reasoning, and data interoperability.

In [ ]:
# Define relationships between tables
g.add((s013, db.relatedTo, lfa1))  # General relationship between tables

# Explicit foreign key relationship
g.add((ns.LIFNR, db.foreignKey, ns.LIFNR))  # S013.LIFNR → LFA1.LIFNR

# Join condition for the relationship
g.add((ns.LIFNR, db.joinCondition, Literal("S013.MANDT = LFA1.MANDT AND S013.LIFNR = LFA1.LIFNR")))

print("✅Table relations have been added to your knowledge graph.")
print(f"👍There are {len(g)} triples in your knowledge graph.")

### Step 6: Serialize the graph in Turtle (Terse RDF Triple Language) format

The Turtle data format is a way of representing data using the RDF. It's a form of serializing RDF data in a human-readable and easy-to-write format. Turtle is defined by the W3C and uses a syntax like the N-Triples format, with added support for prefixes and shorthand notations to make it more concise and readable. Turtle is commonly used for writing and sharing RDF data on the web.

In [ ]:
# Serialize the graph to Turtle format
graph_string = g.serialize(format="turtle")

# Write the Turtle string to a file
with open('./spurchase_tabular.ttl', 'w') as file:
    file.write(graph_string)

print("✅Turtle file has been generated locally.")

### Step 7: Connect to SAP HANA Cloud database

The provided Python script imports database connection modules and initiates a connection to a SAP HANA Cloud instance using the `dbapi` module. The user is prompted to enter their username and password, which are then used to establish a secure connection to the SAP HANA Cloud database. 

In [ ]:
#Set up HANA Cloud Connection to import the ttl file
from hdbcli import dbapi
from dotenv import load_dotenv
import os

load_dotenv()  # take environment variables from .env.

# Get the HANA Cloud username from environment variables
HANA_USER = os.getenv('HANA_VECTOR_USER')
# Get the HANA Cloud password from environment variables
HANA_PASS = os.getenv('HANA_VECTOR_PASS')
# Get the HANA Cloud host from environment variables
HANA_HOST = os.getenv('HANA_VECTOR_HOST')

# Establish connection to SAP HANA Cloud database
conn = dbapi.connect(
    user = HANA_USER,
    password = HANA_PASS,
    address = HANA_HOST,
    port = 443,
)
cursor = conn.cursor()

print("✅HANA Cloud connection is established successfully!")

### Step 8: Import the Turtle file into SAP HANA Cloud
Finally, the Turtle file—containing structured RDF triples that define the semantic relationships between entities—will be imported into the SAP HANA Cloud database, where it will be stored and represented as a knowledge graph. This process enables the underlying data to be queried and analyzed in a graph-based format, allowing for advanced semantic reasoning, relationship exploration, and integration with SAP HANA’s native graph processing capabilities.

In [ ]:
#import the ttl file into SAP HANA Cloud
ttl_filename = "./spurchase_tabular.ttl"
graphname = "spurchase_graph_" + HANA_USER
try:
    with open(ttl_filename, 'r') as ttlfp:
        request_hdrs = ''
        request_hdrs += 'rqx-load-protocol: true' + '\r\n'            # required header for upload protocol
        request_hdrs += 'rqx-load-filename: ' + ttl_filename + '\r\n' # optional header
        request_hdrs += 'rqx-load-graphname: ' + graphname + '\r\n'   # optional header to specify name of the graph, if not provided RDF data will be loaded to internal-default-graph
        conn.cursor().callproc('SPARQL_EXECUTE', (ttlfp.read(), request_hdrs, '', None))

    print("✅Success! The RDF graph has been successfully ingested into SAP HANA Cloud as graph:", graphname)

except Exception as e:
    print("❌Error occurred while ingesting the graph:", str(e))


### Step 9: Validate the Knowledge Graph in SAP HANA Cloud
We will perform queries on the ontologies stored in the SAP HANA Cloud database to explore and validate the semantic structures that have been generated from the underlying tabular data. These queries will help verify that the RDF triples or graph representations correctly model the relationships, hierarchies, and attributes as defined in the original database schema. By doing so, we can ensure that the **graph structure** not only faithfully mirrors the actual tabular data—such as tables, columns, and foreign key constraints—but also captures the **intended semantic relationships** between different entities. This validation step is crucial to confirm the **consistency, completeness, and integrity** of the knowledge graph before it is used for advanced semantic analysis, data integration, or reasoning tasks.

In [ ]:
# View all imported ontologies
validation_query = """
SELECT ?graph (COUNT(*) as ?num_triples)
WHERE {
    GRAPH ?graph { [?p []] }
}
GROUP BY ?graph
ORDER BY ?graph
"""

# Execute the SPARQL query
try:
    resp = cursor.callproc('SPARQL_EXECUTE', (
        validation_query,
        'Accept: application/sparql-results+csv',
        '?',
        None
    ))
    # metadata = resp[3]
    # results = resp[2]
    
    # Print results
    print("Validation Query Results:")
    print(resp[2])
    # print("Response Metadata:", metadata)
except Exception as e:
    raise RuntimeError(f"SPARQL_EXECUTE failed: {e}")


Execute the following code to see all the triples from the imported knowledge graph.

In [ ]:
# View the graph we imported

validation_query = """
SELECT * WHERE {
  GRAPH <""" + graphname + """> {
    ?s ?p ?o
  }
}
"""

# Execute the SPARQL query
try:
    resp = cursor.callproc('SPARQL_EXECUTE', (
        validation_query,
        'Accept: application/sparql-results+csv',
        '?',
        None
    ))
    # metadata = resp[3]
    # results = resp[2]

    # Print results
    print("Validation Query Results:")
    print(resp[2])
    # print("Response Metadata:", metadata)
except Exception as e:
    raise RuntimeError(f"SPARQL_EXECUTE failed: {e}")


### Step 10: Configure AI Core Client
Execute the configuration module below to enable access to SAP’s Generative AI foundation models. Running this code block will automatically handle the necessary setup, including authentication and environment configuration, to ensure seamless connectivity to the Generative AI services.

In [ ]:
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
from gen_ai_hub.proxy.gen_ai_hub_proxy import GenAIHubProxyClient

# Get the AI Core URL from environment variables
URL = os.getenv('AICORE_AUTH_URL')
# Get the AI Core client ID from environment variables
CLIENT_ID = os.getenv('AICORE_CLIENT_ID')
# Get the AI Core client secret from environment variables
CLIENT_SECRET = os.getenv('AICORE_CLIENT_SECRET')
# Get the AI Core client ID from environment variables
RESOURCE_GROUP = os.getenv('AICORE_RESOURCE_GROUP')
# Get the AI Core client secret from environment variables
API_URL = os.getenv('AICORE_BASE_URL')

# Set up the AICoreV2Client
ai_core_client = AICoreV2Client(base_url=API_URL,
                            auth_url=URL,
                            client_id=CLIENT_ID,
                            client_secret=CLIENT_SECRET,
                            resource_group=RESOURCE_GROUP)

# Initialize GenAIHub proxy client
proxy_client = GenAIHubProxyClient(ai_core_client=ai_core_client)
print("✅AI Core Client connection is established successfully!")

The large language model (LLM) is initialized as an instance of ChatBedrock using the **anthropic--claude-3.5-sonnet** model. This instance serves as the conversational interface, enabling the generation of context-aware responses and facilitating interactions in a chat-like environment.

In [ ]:
from gen_ai_hub.proxy.langchain.amazon import ChatBedrock
# Initialize the ChatBedrock client with the proxy client
anthropic = ChatBedrock(
    model_name="anthropic--claude-3.7-sonnet",
    proxy_client=proxy_client # Pass the proxy client to ChatBedrock
)
print("✅LLM model connection is established successfully!")

### Step 11: Extract Relevant Metadata using SPARQL
This script executes a SPARQL query against a knowledge graph stored in SAP HANA Cloud, parses the XML response, and extracts the resulting RDF triples (subject, predicate, object) into a Python list called metadata.

In [ ]:
from xml.etree import ElementTree as ET

# Execute SPARQL query to get all relevant triples
sparql_query = """
SELECT * WHERE {
  GRAPH <""" + graphname + """> {
    ?s ?p ?o
  }
}
"""
    
try:
    cursor = conn.cursor()
    resp = cursor.callproc('SPARQL_EXECUTE', (sparql_query, 'Metadata headers describing Input and/or Output', '?', None))
    
    if resp and len(resp) >= 3 and resp[2]:
        # Parse the XML response
        xml_response = resp[2]
        try:
            root = ET.fromstring(xml_response)
            results = []
            
            for result in root.findall('.//{http://www.w3.org/2005/sparql-results#}result'):
                row = {}
                for binding in result:
                    var_name = binding.attrib['name']
                    value = binding[0]  # uri or literal
                    if value.tag.endswith('uri'):
                        row[var_name] = value.text
                    elif value.tag.endswith('literal'):
                        row[var_name] = value.text
                results.append(row)
        except ET.ParseError as e:
            print(f"Error parsing XML: {e}")
        # results = parse_sparql_results(xml_response)
        
        # Convert to our standard format
        metadata = []
        for row in results:
            metadata.append({
                's': row.get('s', ''),
                'p': row.get('p', ''),
                'o': row.get('o', '')
            })

    print(f"✅SPARQL query executed successfully! Retrieved {len(metadata)} triples.")

    # Print the metadata
    for item in metadata:
        print(item)

except Exception as e:
    print(f"Error executing SPARQL query: {e}")


### Step 12: Analyze the Metadata and Natural Language Question
This code snippet is designed to analyze a natural language question by leveraging metadata from a knowledge graph and using a large language model (LLM) (via LangChain and Anthropic) to extract relevant SQL components (tables, columns, filters, joins, etc.).

Some other questions you can ask: 
1. What is the risk score for purchasing material 'MAT0151' from vendor '1011'?
2. Find the vendors' name whose risk score is higher than 34 for the material MAT0151.

In [ ]:
from langchain_core.prompts import PromptTemplate
# Analyze the metadata to identify tables, columns, and relationships
# Convert metadata to a format the LLM can understand
metadata_str = "\n".join([f"{item['s']} {item['p']} {item['o']}" for item in metadata])
question = "Find the vendors's name whose risk score is higher than 34 for the material MAT0151."

prompt_template = """Given the following RDF metadata about database tables and columns, analyze the user's question and identify:
1. The main table(s) involved with their schema (SPURCHASE)
2. The columns needed (including any aggregation functions)
3. Any filters or conditions
4. Any joins required

Important Rules:
- Always include the schema name (SPURCHASE) before table names
- When using GROUP BY, include the grouping columns in SELECT
- Never include any explanatory text in the SQL output
- For country codes like Germany, use 'DE' in filters

For each column, include:
- The column name (prefix with table alias if needed)
- Any aggregation function (AVG, COUNT, etc.)
- Any filter conditions
- Whether it's a grouping column

For tables, include:
- The full table name with schema (e.g., SPURCHASE.S013)
- Any relationships to other tables


Metadata:
{metadata}

Question: {question}

Return your analysis in this exact format (without any additional explanations):
Tables: [schema1.table1, schema2.table2]
Columns: [column1, column2, column3, column with aggregations like AVG(RISK1)]
Filters: [filter condition1,filter condition2]
Joins: [join condition1, join condition2]
GroupBy: [columns1, columns2]
"""

prompt = PromptTemplate.from_template(prompt_template).invoke({
    "metadata": metadata_str,
    "question": question
})

# We'll use the LLM to extract the key components
analysis = anthropic.invoke(prompt)
print(analysis.content)
# return parse_analysis(analysis.content)

### Step 13: Parse the LLM Response
This code parses and structures the response from a Large Language Model (LLM), which analyzed a natural language question and metadata to return components for an SQL query (e.g., tables, columns, filters). 

In [ ]:
# Define a library to parse the analysis content
components = {
    "tables": [],
    "columns": [],
    "filters": [],
    "joins": [],
    "group_by": []
}

# Remove any "Explanation:" text
analysis.content = analysis.content.split("Explanation:")[0].strip()

# Parse each section
current_section = None
for line in analysis.content.split('\n'):
    line = line.strip()
    if not line:
        continue
        
    if line.startswith('Tables:'):
        current_section = 'tables'
        tables = line.split(':')[1].strip()
        components['tables'] = [t.strip() for t in tables.split(',') if t.strip()]
    elif line.startswith('Columns:'):
        current_section = 'columns'
        cols = line.split(':')[1].strip()
        for col_part in cols.split(','):
            col_part = col_part.strip()
            if col_part:
                if '(' in col_part and ')' in col_part:
                    agg = col_part.split('(')[0].strip().upper()
                    col = col_part.split('(')[1].split(')')[0].strip()
                    components['columns'].append((agg, col))
                else:
                    components['columns'].append((None, col_part))
    elif line.startswith('Filters:'):
        current_section = 'filters'
        filters = line.split(':')[1].strip()
        components['filters'] = [f.strip() for f in filters.split(',') if f.strip()]
    elif line.startswith('Joins:'):
        current_section = 'joins'
        joins = line.split(':')[1].strip()
        components['joins'] = [j.strip() for j in joins.split(',') if j.strip()]
    elif line.startswith('GroupBy:'):
        current_section = 'group_by'
        group_bys = line.split(':')[1].strip()
        components['group_by'] = [g.strip() for g in group_bys.split(',') if g.strip()]
    elif current_section:
        # Handle multi-line sections
        if current_section == 'tables':
            components['tables'].extend([t.strip() for t in line.split(',') if t.strip()])
        elif current_section == 'columns':
            for col_part in line.split(','):
                col_part = col_part.strip()
                if col_part:
                    if '(' in col_part and ')' in col_part:
                        agg = col_part.split('(')[0].strip().upper()
                        col = col_part.split('(')[1].split(')')[0].strip()
                        components['columns'].append((agg, col))
                    else:
                        components['columns'].append((None, col_part))
        elif current_section == 'filters':
            components['filters'].extend([f.strip() for f in line.split(',') if f.strip()])
        elif current_section == 'joins':
            components['joins'].extend([j.strip() for j in line.split(',') if j.strip()])
        elif current_section == 'group_by':
            components['group_by'].extend([g.strip() for g in line.split(',') if g.strip()])

# Ensure schema is included in table names
components['tables'] = [f"SPURCHASE.{t.split('.')[-1]}" if '.' not in t else t for t in components['tables']]

# Ensure grouping columns are included in SELECT - CORRECTED VERSION
for group_col in components['group_by']:
    # Check if this exact (None, group_col) pair exists
    col_exists = any(col == (None, group_col) for col in components['columns'])
    # Check if group_col appears in any non-aggregated column reference
    col_part_of_ref = any(group_col in col[1] for col in components['columns'] if col[0] is None)
    
    if not col_exists and not col_part_of_ref:
        components['columns'].append((None, group_col))
print(components)
# return components

### Step 14: Generate SQL Query
This code snipet takes the structured components extracted from a Large Language Model (LLM) and generates a clean, valid SQL query. 

In [ ]:
# Generate clean SQL query from the analyzed components"""
# Validate components
if not components["tables"]:
    raise ValueError("No tables identified for SQL generation")

# Clean all components first
def clean_component(component):
    return component.replace('[', '').replace(']', '').strip()

# Build SELECT clause - ensure GROUP BY columns are included
select_parts = []

# First add all GROUP BY columns to SELECT if they're not already there
for group_col in components.get("group_by", []):
    group_col = clean_component(group_col)
    if not any(col[1] == group_col for col in components["columns"] if col[0] is None):
        select_parts.append(group_col)

# Then add the requested columns
for agg, col in components["columns"]:
    col = clean_component(col)
    if not col:
        continue
    if agg:
        select_parts.append(f"{agg}({col}) AS {agg}_{col}")
    else:
        if col not in select_parts:  # Don't add duplicates
            select_parts.append(col)

if not select_parts:  # Default to all columns if none specified
    select_parts.append("*")

select_clause = ", ".join(select_parts[1:])
# print("SELECT BEFORE "+select_clause)
# print(select_parts)

# Build FROM clause
from_table = clean_component(components["tables"][0])
from_clause = from_table

# Add joins only if they exist and are not empty
join_clauses = []
for join in components.get("joins", []):
    clean_join = clean_component(join)
    if clean_join and clean_join != 'INNER JOIN':
        join_clauses.append(f"INNER JOIN SPURCHASE.LFA1 ON {clean_join}")
# print("INNER JOIN "+clean_join)

# Build WHERE clause
where_clauses = []
for filter_cond in components.get("filters", []):
    clean_filter = clean_component(filter_cond)
    if clean_filter:
        where_clauses.append(clean_filter)

where_clause = " AND ".join(where_clauses) if where_clauses else ""
where_clause = where_clause.replace(",", " AND")
# print("WHERE CLAUSE "+where_clause)

# Build GROUP BY clause
group_by_columns = [clean_component(g) for g in components.get("group_by", []) if clean_component(g)]
group_by_clause = ", ".join(group_by_columns) if group_by_columns else ""

# Construct the SQL
sql = f"SELECT {select_clause} FROM {from_clause}"

if join_clauses:
    sql += " " + " ".join(join_clauses)

if where_clause:
    sql += f" WHERE {where_clause}"

if group_by_clause:
    sql += f" GROUP BY {group_by_clause}"

# Final formatting
sql = sql.strip()
if not sql.endswith(';'):
    sql += ';'

print(sql)
#return sql

### Step 15: Execute the SQL Query
This code executes a SQL query using a database connection (conn) and processes the result into a clean, tabular format using Pandas.

In [ ]:
import pandas as pd

# Execute the generated SQL query and return results
try:
    cursor.execute(sql)
    columns = [desc[0] for desc in cursor.description]
    rows = cursor.fetchall()
    results = pd.DataFrame(rows, columns=columns)
    print(results)
except Exception as e:
    print(f"Error executing SQL query: {e}")
finally:
    cursor.close()


### Step 16: Generate and Return the Final Response
This code generates a natural language explanation of the SQL query results using a large language model (LLM). 

In [ ]:
# Generate a natural language response from the query results
if results.empty:
    print("❌No results found for your query!")

prompt_template = """Convert the following query results into a natural language response to the user's question. 
Keep the response concise but informative. Include relevant numbers and comparisons where appropriate.

Question: {question}

Results:
{results}

Response:
"""

prompt = PromptTemplate.from_template(prompt_template).invoke({
    "question": question,
    "results": results.to_string()
})

response = anthropic.invoke(prompt)
print(response.content)
print("💯This is the end of the demo. Thank you for your attention!")